In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import MeanShift, estimate_bandwidth

# Synthetic GPS crime data for Sfax (approx coordinate bounds)
np.random.seed(42)

# Hotspot clusters
medina = np.random.normal(loc=(34.742, 10.760), scale=0.002, size=(120, 2))
sakiet = np.random.normal(loc=(34.770, 10.740), scale=0.0025, size=(80, 2))
thyna  = np.random.normal(loc=(34.720, 10.700), scale=0.002, size=(60, 2))
el_ain = np.random.normal(loc=(34.755, 10.725), scale=0.003, size=(50, 2))

coords = np.vstack([medina, sakiet, thyna, el_ain])
df = pd.DataFrame(coords, columns=['lat', 'lon'])

# Generate random scattered crime noise across Sfax
noise_lat = np.random.uniform(34.70, 34.80, size=60)   # latitude spread
noise_lon = np.random.uniform(10.65, 10.80, size=60)   # longitude spread
noise = np.column_stack((noise_lat, noise_lon))

# Append noise to existing hotspot data
coords_with_noise = np.vstack([coords, noise])
df = pd.DataFrame(coords_with_noise, columns=['lat', 'lon'])

# Re-run clustering on the data WITH noise
bandwidth = estimate_bandwidth(df[['lat', 'lon']], quantile=0.15)
ms = MeanShift(bandwidth=bandwidth)
df['cluster'] = ms.fit_predict(df[['lat','lon']])
centers = ms.cluster_centers_

# Export to CSV
df.to_csv("sfax_crime_data.csv", index=False)

In [2]:
import folium
from folium.plugins import HeatMap

# Center of Sfax
m = folium.Map(location=[34.74, 10.76], zoom_start=12)

# Color palette
colors = [
    "red", "blue", "green", "purple", 
    "orange", "darkred", "cadetblue", "black"
]


# Plot cluster points
for cluster_id in df['cluster'].unique():
    cluster_data = df[df['cluster'] == cluster_id]
    for _, row in cluster_data.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=3,
            color=colors[cluster_id % len(colors)],
            fill=True,
            fill_opacity=0.6
        ).add_to(m)

# Plot cluster centers
for c in centers:
    folium.Marker(
        location=[c[0], c[1]],
        icon=folium.Icon(color='black', icon='star')
    ).add_to(m)

m

m.save("sfax_crime_hotspots.html")
print("✅ Interactive map saved as sfax_crime_hotspots.html")

✅ Interactive map saved as sfax_crime_hotspots.html


In [3]:
crime_map = folium.Map(location=[34.74, 10.76], zoom_start=12)

HeatMap(df[['lat','lon']].values, radius=12, blur=10).add_to(crime_map)

crime_map.save("sfax_crime_hotspots_heatmap.html")
print("✅ Interactive heatmap saved as sfax_crime_hotspots.html")

✅ Interactive heatmap saved as sfax_crime_hotspots.html
